In [1]:
import os

In [2]:
%pwd

'/Users/naveendas/Desktop/AI_projects/e2e_mlflow/research'

In [3]:
os.chdir("../")
%pwd

'/Users/naveendas/Desktop/AI_projects/e2e_mlflow'

In [ ]:
import dagshub
dagshub.init(repo_owner='Naveen-das09', repo_name='e2e_mlflow', mlflow=True)

In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    


In [20]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories, save_json


In [21]:


class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name            
        )

        return model_evaluation_config


In [22]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import dagshub

In [23]:


class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config=config
    
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_absolute_error(actual,pred))
        mse = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mse, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        dagshub.init(repo_owner='Naveen-das09', repo_name='e2e_mlflow', mlflow=True)

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y,predicted_qualities)

            # saving metrics as local

            scores = {"rmse" : rmse, "mae" : mae, "r2" : r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2",r2)

            mlflow.sklearn.log_model(
                model,
                "model",
                registered_model_name="ElasticNetModel"                
            )

In [24]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-08-03 19:32:00,802:INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-03 19:32:00,804:INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-03 19:32:00,806:INFO: common: yaml file: schema.yaml loaded successfully]
[2024-08-03 19:32:00,807:INFO: common: created directory at: artifacts]
[2024-08-03 19:32:00,808:INFO: common: created directory at: artifacts/model_evaluation]


Initialized MLflow to track repo "Naveen-das09/e2e_mlflow"

[2024-08-03 19:32:01,853:INFO: helpers: Initialized MLflow to track repo "Naveen-das09/e2e_mlflow"]


Repository Naveen-das09/e2e_mlflow initialized!

[2024-08-03 19:32:01,854:INFO: helpers: Repository Naveen-das09/e2e_mlflow initialized!]
[2024-08-03 19:32:20,182:WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='dagshub.com', port=443): Read timed out. (read timeout=120)")': /Naveen-das09/e2e_mlflow.mlflow/api/2.0/mlflow/runs/create]
[2024-08-03 19:32:20,758:INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


Registered model 'ElasticNetModel' already exists. Creating a new version of this model...
2024/08/03 19:32:29 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticNetModel, version 2
Created version '2' of model 'ElasticNetModel'.
